In [1]:
# https://www.kaggle.com/joparga3/in-depth-skewed-data-classif-93-recall-acc-now/notebook

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


**Why do I use this kernel?**

I have a deeply interest in find out if a transaction is a fraud or not. 
About the Data set: 
*     it contains transactions made by credit cards in September 2013 by european cardholders.
*     It's highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.
*     It contains only numerical input variables which are the result of a PCA transformation (confidential data).

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Read data
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [4]:
# Check the columns and general structure
df.head()

As we can see, the data is confidential, we don't know the column names.

In [ ]:
# See if the data is balanced or unbalanced

In [5]:
from matplotlib.legend_handler import HandlerBase
from matplotlib.text import Text
class TextHandler(HandlerBase):
    def create_artists(self, legend, tup ,xdescent, ydescent, width, height, fontsize,trans):
        tx = Text(width/2.,height/2,tup[0], fontsize=fontsize,
                  ha="center", va="center", color=tup[1], fontweight="bold")
        return [tx]
    
ax = sns.countplot(x="Class", data=df)
plt.title("Frauds vs Not. Frauds")
plt.xlabel("Target")
plt.ylabel("Count")
sns.set(rc = {'figure.figsize':(8,8)})
#Add value counts to each bar
for p in ax.patches:
   ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))

# Add right legend
handltext = ["0", "1"]
labels = ["Not Fraud", "Fraud"]

t = ax.get_xticklabels()
labeldic = dict(zip(handltext, labels))
labels = [labeldic[h.get_text()]  for h in t]
handles = [(h.get_text(),c.get_fc()) for h,c in zip(t,ax.patches)]

ax.legend(handles, labels, handler_map={tuple : TextHandler()}) 

